In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Iris copy.csv')
df.drop(columns = ['Id'], inplace= True)

type = []
for i in df.Species:
    if i == 'Iris-setosa':
        i = 0
    elif i == 'Iris-versicolor':
        i = 1
    else:
        i = 2

    type.append(i)

df['type'] = type

df.drop(columns= ['Species'], inplace= True)

df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [3]:
def gini_index(target):
    target = pd.DataFrame(target)
    target = target.iloc[:, -1]
    feature_labels = np.unique(target)
    g = 0
    for i in feature_labels:
        probability = len(target[target == i]) / len(target)
        g += probability**2
    return 1 - g

def information_gain(parent, left_child, right_child):
    parent = pd.DataFrame(parent)
    left_child = pd.DataFrame(left_child)
    right_child = pd.DataFrame(right_child)
    wl = len(left_child) / len(parent)
    wr = len(right_child) / len(parent)
    info_gain = gini_index(parent.iloc[:, -1]) - ((wl * gini_index(left_child.iloc[:, -1])) + (wr * gini_index(right_child.iloc[:, -1])))
    return info_gain

In [4]:
def first_split(dataset):
    num_samples, num_features = np.shape(dataset)
    first_split_dict = {}
    for feature_index in range(num_features-1):
        value = dataset.iloc[:, feature_index]
        for threshold in np.unique(value):
            dataset_left = np.array([row for row in value if row <= threshold])
            dataset_right = np.array([row for row in value if row > threshold])
            parent = np.concatenate((dataset_left,dataset_right), axis=0)
            if len(dataset_left) > 0 and len(dataset_right) > 0:
                y, y_left, y_right = parent, dataset_left, dataset_right
                # info_gain[threshold] = information_gain(y, y_left, y_right)
                first_split_dict[threshold] = (y, y_left, y_right)
    return first_split_dict


def split(dataset):
    split_dict = {}
    for threshold in np.unique(dataset):
        dataset_left = np.array([row for row in value if row <= threshold])
        dataset_right = np.array([row for row in value if row > threshold])
        parent = np.concatenate((dataset_left,dataset_right), axis=0)
        if len(dataset_left) > 0 and len(dataset_right) > 0:
            y, y_left, y_right = parent, dataset_left, dataset_right
            # info_gain[threshold] = information_gain(y, y_left, y_right)
            split_dict[threshold] = (y, y_left, y_right)
    return split_dict


def best_threshold(dictionary):
    info = {}
    for i in dictionary.keys():
        for j in range(3):
            each_info = information_gain(dictionary[i][0], dictionary[i][1], dictionary[i][2])
            info[i] = each_info
    max_key = max(info, key=info.get)

    return max_key


def best_split(dataset, threshold):
    num_samples, num_features = np.shape(dataset)
    best_split_dict = {}
    for feature_index in range(num_features-1):
        value = dataset.iloc[:, feature_index]

        dataset_left = np.array([row for row in value if row <= threshold])
        dataset_right = np.array([row for row in value if row > threshold])
        best_split_dict[feature_index] = (dataset_left, dataset_right)
            
    return best_split_dict

def dataframe(dataset, bestSplit):
    for i in range(len(bestSplit)):
        if len(bestSplit[i][0]) > 0 and len(bestSplit[i][1]) > 0:
            left = bestSplit[i][0]
            right = bestSplit[i][1]

            dataset_left = dataset.loc[dataset[col_names[i]].isin(left)]
            dataset_right = dataset.loc[dataset[col_names[i]].isin(right)]

    return dataset_left, dataset_right

In [5]:
col_names = []
for i in df:
    col_names.append(i)

In [6]:
dict_1 = first_split(df)
thresh_1 = best_threshold(dict_1)
best_split_1 = best_split(df, thresh_1)

In [7]:
len(best_split_1)

4

In [8]:
df2, df3 = dataframe(df, best_split_1)

In [61]:
information_gain(df, df2, df3)

0.19540229885057475

In [9]:
dict_2 = first_split(df2)
thresh_2 = best_threshold(dict_2)
best_split_2 = best_split(df2, thresh_2)
df21, df22 = dataframe(df2, best_split_2)
# gini_index(df22['type'])
df21

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
9,4.9,3.1,1.5,0.1,0
12,4.8,3.0,1.4,0.1,0
13,4.3,3.0,1.1,0.1,0
32,5.2,4.1,1.5,0.1,0
34,4.9,3.1,1.5,0.1,0
37,4.9,3.1,1.5,0.1,0


In [62]:
information_gain(df2, df21,df22)

0.0

In [10]:
best_split_2[3][0]

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [11]:
df3

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
15,5.7,4.4,1.5,0.4,0
16,5.4,3.9,1.3,0.4,0
17,5.1,3.5,1.4,0.3,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [12]:
dict_3 = first_split(df3)
thresh_3 = best_threshold(dict_3)
best_split_3 = best_split(df3, thresh_3)

In [13]:
df4, df5 = dataframe(df3, best_split_3)

In [14]:
df5

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [15]:
dict_5 = first_split(df5)
thresh_5 = best_threshold(dict_5)
best_split_5 = best_split(df5, thresh_5)

In [16]:
df6, df7 = dataframe(df5, best_split_5)

In [17]:
gini_index(df6.iloc[:, -1])

0.47206611570247936

In [207]:
df7

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
56,6.3,3.3,4.7,1.6,1
61,5.9,3.0,4.2,1.5,1
65,6.7,3.1,4.4,1.4,1
66,5.6,3.0,4.5,1.5,1
70,5.9,3.2,4.8,1.8,1
75,6.6,3.0,4.4,1.4,1
77,6.7,3.0,5.0,1.7,1


In [18]:
dict_6 = first_split(df6)
thresh_6 = best_threshold(dict_6)
best_split_6 = best_split(df6, thresh_6)

In [19]:
df61, df62 = dataframe(df6, best_split_6)

In [20]:
df61

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
53,5.5,2.3,4.0,1.3,1
57,4.9,2.4,3.3,1.0,1
59,5.2,2.7,3.9,1.4,1
60,5.0,2.0,3.5,1.0,1
62,6.0,2.2,4.0,1.0,1
67,5.8,2.7,4.1,1.0,1
68,6.2,2.2,4.5,1.5,1
69,5.6,2.5,3.9,1.1,1
72,6.3,2.5,4.9,1.5,1
79,5.7,2.6,3.5,1.0,1


In [21]:
gini_index(df61['type'])

0.451171875

In [22]:
df62

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
54,6.5,2.8,4.6,1.5,1
55,5.7,2.8,4.5,1.3,1
58,6.6,2.9,4.6,1.3,1
63,6.1,2.9,4.7,1.4,1
64,5.6,2.9,3.6,1.3,1
71,6.1,2.8,4.0,1.3,1
73,6.1,2.8,4.7,1.2,1
74,6.4,2.9,4.3,1.3,1
76,6.8,2.8,4.8,1.4,1
78,6.0,2.9,4.5,1.5,1


In [23]:
dict_7 = first_split(df7)
thresh_7 = best_threshold(dict_7)
best_split_7 = best_split(df7, thresh_7)

In [24]:
df71, df72 = dataframe(df7, best_split_7)

In [25]:
df71

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
61,5.9,3.0,4.2,1.5,1
66,5.6,3.0,4.5,1.5,1
75,6.6,3.0,4.4,1.4,1
77,6.7,3.0,5.0,1.7,1
84,5.4,3.0,4.5,1.5,1
88,5.6,3.0,4.1,1.3,1
91,6.1,3.0,4.6,1.4,1
95,5.7,3.0,4.2,1.2,1
102,7.1,3.0,5.9,2.1,2
104,6.5,3.0,5.8,2.2,2


In [26]:
gini_index(df71['type'])

0.48

In [27]:
df72

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
56,6.3,3.3,4.7,1.6,1
65,6.7,3.1,4.4,1.4,1
70,5.9,3.2,4.8,1.8,1
85,6.0,3.4,4.5,1.6,1
86,6.7,3.1,4.7,1.5,1
100,6.3,3.3,6.0,2.5,2
109,7.2,3.6,6.1,2.5,2


In [28]:
dict_72 = first_split(df72)
thresh_72 = best_threshold(dict_72)
best_split_72 = best_split(df72, thresh_72)
df721, df722 = dataframe(df72, best_split_72)
df722

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
56,6.3,3.3,4.7,1.6,1
70,5.9,3.2,4.8,1.8,1
85,6.0,3.4,4.5,1.6,1
100,6.3,3.3,6.0,2.5,2
109,7.2,3.6,6.1,2.5,2
110,6.5,3.2,5.1,2.0,2
115,6.4,3.2,5.3,2.3,2
117,7.7,3.8,6.7,2.2,2


In [29]:
dict_722 = first_split(df722)
thresh_722 = best_threshold(dict_722)
best_split_722 = best_split(df722, thresh_722)
df7221, df7222 = dataframe(df722, best_split_722)

# rightchild = gini_index(df7222.iloc[:, -1])
# leftchild = gini_index(df7221['type'])
# parent = gini_index(df722['type'])

# wright = len(df7222) / len(df722)
# wleft = len(df7221) / len(df722)

# parent - (wleft*leftchild + wright*rightchild)
information_gain(df722, df7221, df7222)

0.015123456790123557

In [30]:
dict_7222 = first_split(df7222)
thresh_7222 = best_threshold(dict_7222)
best_split_7222 = best_split(df7222, thresh_7222)
df72221, df72222 = dataframe(df7222, best_split_7222)
# gini_index(df72222.iloc[:, -1])
information_gain(df7222, df72221, df72222)

0.0033333333333331883

In [39]:
def build_tree(dataset, max_depth = 3):
    tree_split = first_split(dataset)
    tree_threshhold = best_threshold(tree_split)
    best_tree_split = best_split(dataset, tree_threshhold)
    left_child, right_child = dataframe(dataset, best_tree_split)

    left_child = pd.DataFrame(left_child)
    right_child = pd.DataFrame(right_child)

    if gini_index(left_child.iloc[:, -1]) > 0 or gini_index(right_child.iloc[:, -1]) > 0:
        return left_child, right_child

    elif gini_index(left_child.iloc[:, -1]) == 0 and gini_index(right_child.iloc[:, -1]) == 0:
            
        return dataset

# def DecisionTree():
    

In [60]:
from collections import Counter

def classify(data):
    counts = Counter(data.iloc[:, -1])
    classified = max(counts, key=counts.get)

    return classified    

In [64]:
class Node():
    def __init__(self, threshold, left_branch, right_branch):

        self.threshold = threshold
        self.left_branch = left_branch
        self.right_branch = right_branch

class Leaf():
    def __init__(self, data):
        self.classification = classify(data)



In [ ]:
def build_tree(dataset, max_depth):
    tree_split = first_split(dataset)
    tree_threshhold = best_threshold(tree_split)
    best_tree_split = best_split(dataset, tree_threshhold)
    left_child, right_child = dataframe(dataset, best_tree_split)

    left_child = pd.DataFrame(left_child)
    right_child = pd.DataFrame(right_child)

    
    depth = 0
    while depth < max_depth:
        if gini_index(left_child.iloc[:, -1]) == 0:
            return classify(left_child)
        elif gini_index(left_child.iloc[:, -1]) > 0:
            

            return left_child, right_child

    if gini_index(left_child.iloc[:, -1]) > 0 or gini_index(right_child.iloc[:, -1]) > 0:
        return left_child, right_child

    elif gini_index(left_child.iloc[:, -1]) == 0 and gini_index(right_child.iloc[:, -1]) == 0:
            
        return dataset

# def DecisionTree():
    

In [40]:
tree1 = build_tree(df)

In [41]:
right = pd.DataFrame(tree1[1])

In [42]:
right

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
15,5.7,4.4,1.5,0.4,0
16,5.4,3.9,1.3,0.4,0
17,5.1,3.5,1.4,0.3,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [43]:
gini_index(right)

0.6093935790725328

In [44]:
tree2 = build_tree(right)
right2 = tree2[1]
right2

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [45]:
tree3 = build_tree(right2)
left3 = tree3[0]
right3 = tree3[1]
right3

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
56,6.3,3.3,4.7,1.6,1
61,5.9,3.0,4.2,1.5,1
65,6.7,3.1,4.4,1.4,1
66,5.6,3.0,4.5,1.5,1
70,5.9,3.2,4.8,1.8,1
75,6.6,3.0,4.4,1.4,1
77,6.7,3.0,5.0,1.7,1


In [38]:
left3

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,type
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
55,5.7,2.8,4.5,1.3,1
57,4.9,2.4,3.3,1.0,1
58,6.6,2.9,4.6,1.3,1
59,5.2,2.7,3.9,1.4,1
60,5.0,2.0,3.5,1.0,1
62,6.0,2.2,4.0,1.0,1
63,6.1,2.9,4.7,1.4,1
64,5.6,2.9,3.6,1.3,1
